In [1]:
import numpy as np
import pandas as pd
from main import MLP_Classifier,Layer
from sklearn.datasets import make_classification
import optuna
from sklearn.metrics import accuracy_score,precision_score,recall_score
from sklearn.model_selection import train_test_split

/opt/python/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
np.random.seed(42)
# Generate  dataset
X, Y = make_classification(
    n_samples=1000,     
    n_features=4,       
    n_redundant=0,      
    n_clusters_per_class=1,
    flip_y=0.1,         # Add label noise
    class_sep=1.0,      # Class separation
    n_classes=2,      # nb classes
)


noise = np.random.normal(0, 0.5, X.shape)
X = X + noise
X=pd.DataFrame(X)
Y=pd.Series(Y)
if len(np.unique(Y))==2:
   Y=pd.DataFrame(Y)
else: 
   Y=pd.get_dummies(Y).astype(int)


determine objective function to optimise :  minimise cross entropy (in pdf maximise log-likelihood).

for example we optimise over batch size, learning rate and dropout (one of the most important parameters in NN).

we could do also on layers but computantionally expensive for large datasets.


In [3]:



def objective(trial):
    # Define hyperparameter search space
    batch_size = trial.suggest_int("batch_size", 500, 800)
    alpha = trial.suggest_float("alpha", 0.01, 0.1)
    dropout_rate = trial.suggest_float("dropout", 0.5, 0.9)

    model = MLP_Classifier(
        (
            (
                Layer(
                    nb_neurons=20,
                    activation_function="relu",
                    regul=("l2", 0.1),
                    initial="he",
                    batchnorm=True
                ),
                Layer(
                    nb_neurons=10,
                    activation_function="relu",
                    regul=("l2", 0.1),
                    initial="he",
                ),
                Layer(
                    nb_neurons=30,
                    activation_function="relu",
                    regul=("dropout", dropout_rate),
                    initial="he",
                ),
            )
        ),
        max_iter=2000,
        thr=1e-5,
        alpha=alpha,
        seed=123,
        batch_size=batch_size,
        verbose=False,
        optim="adam"
    )

    model.train(X, Y)

    score = model.loss(Y)  # need to do on val set

    return score


storage = "sqlite:///optuna_mlpsoftmax.db"
study = optuna.create_study(
    direction="minimize", study_name="MLP", storage=storage, load_if_exists=True
)  # 'minimize' for loss functions
study.optimize(objective, n_trials=1)

print("Best Hyperparameters:", study.best_params)


[I 2025-10-25 10:11:50,434] Using an existing study with name 'MLP' instead of creating a new one.


Don't forget to normalise input data and think about Batch normalisations


[I 2025-10-25 10:11:58,773] Trial 4 finished with value: 0.6901697395231332 and parameters: {'batch_size': 739, 'alpha': 0.07146452413642196, 'dropout': 0.7221811614090188}. Best is trial 4 with value: 0.6901697395231332.


Best Hyperparameters: {'batch_size': 739, 'alpha': 0.07146452413642196, 'dropout': 0.7221811614090188}


In [4]:
best_results={"best value" : study.best_trial.values,"params": study.best_trial.params}
best_results


{'best value': [0.6901697395231332],
 'params': {'batch_size': 739,
  'alpha': 0.07146452413642196,
  'dropout': 0.7221811614090188}}

run model on optimised parameters

In [6]:


X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.25, random_state=42)

model = MLP_Classifier(
    (
        (
            Layer(
                nb_neurons=20,
                activation_function="relu",
                regul=("l2", 0.1),
                initial="he",
                batchnorm=True
         
            ),
            Layer(
                nb_neurons=10,
                activation_function="relu",
                regul=("l2", 0.1),
                initial="he",
                
            ),
            Layer(
                nb_neurons=30,
                activation_function="relu",
                regul=("dropout", best_results["params"]["dropout"]),
                initial="he",
                
            ),
        )
    ),
    max_iter=2000,
    thr=1e-5,
    alpha=best_results["params"]["alpha"],
    seed=123,
    batch_size=best_results["params"]["batch_size"],
    verbose=True,
    optim="adam",
    nb_epochs_early_stopping=50
)

fct=accuracy_score

model.train(X_train, y_train,X_test,y_test,fct)

print(f"final {fct.__name__}", accuracy_score(model.predict(X_test), y_test))


-------------------------------------------------------------------------
iteration 0 : TRAIN accuracy_score  : 0.7466666666666667, loss : 0.5850563731290825
iteration 0 : TEST accuracy_score  : 0.744, loss : 0.5982165217910403
-------------------------------------------------------------------------
iteration 50 : TRAIN accuracy_score  : 0.8146666666666667, loss : 0.41649394059028344
iteration 50 : TEST accuracy_score  : 0.852, loss : 0.39309214063598324
early stopping at epoch 79
final accuracy_score 0.804
